# Chapter 4

In [1]:
import sympy as sp
from sympy import symbols, sqrt, init_printing
from sympy_equation import Eqn, solve, table_of_expressions
from ambiance import Atmosphere
import numpy as np
import pint
import pygasflow
from pygasflow import *
from pygasflow.atd import *
init_printing()
pygasflow.defaults.solver_to_dict = True

ureg = pint.UnitRegistry()
# use "~P" to format units with unicode
ureg.formatter.default_format = "~"
pygasflow.defaults.pint_ureg = ureg
K = ureg.K
m = ureg.m
km = ureg.km
s = ureg.s
J = ureg.J
W = ureg.W
kg = ureg.kg
deg = ureg.deg
atm = ureg.atm

## P 4.1

In [2]:
from scipy.optimize import bisect

In [3]:
L = 0.1 * m
v = 3000 * m / s
Sr = 0.2

In [4]:
t_res = L / v
t_res

<Quantity(3.33333333e-05, 'second')>

In [5]:
def func(t_ref):
    return Sr - Strouhal(t_res, t_ref * s)

t_ref_lim = bisect(func, a=1e-12, b=1e06)
t_ref_lim *= s
t_ref_lim

<Quantity(0.000166666666, 'second')>

It must be $t_{ref} \ge 1.67 \cdot 10^{-04} s$

## P 4.2

In [6]:
T = 500 * K

In [7]:
mu_s = viscosity_air_southerland(T)
mu_s

<Quantity(2.67053335e-05, 'kilogram / meter / second')>

In [8]:
mu_1 = 0.702e-07 * T.magnitude * (kg / (m * s))
mu_1

<Quantity(3.51e-05, 'kilogram / meter / second')>

In [9]:
mu_2 = viscosity_air_power_law(T)
mu_2

<Quantity(2.63768369e-05, 'kilogram / meter / second')>

## P 4.3

In [10]:
T = 500 * K

In [11]:
k_han = thermal_conductivity_hansen(T)
k_han

<Quantity(0.0364091788, 'watt / meter / kelvin')>

In [12]:
k_1 = 9.572e-05 * T.magnitude * (W / (m * K))
k_1

<Quantity(0.04786, 'watt / meter / kelvin')>

In [13]:
k_2 = thermal_conductivity_power_law(T)
k_2

<Quantity(0.0369625273, 'watt / meter / kelvin')>

## P 4.4

Using real gas model:

In [14]:
from CoolProp.CoolProp import PropsSI
T = 500 * K
p = 1 * atm

In [15]:
T = T.magnitude
p = p.to("Pa").magnitude

# specific heat at constant pressure (J/kg/K)
Cp = PropsSI('Cpmass', 'T', T, 'P', p, 'Air')
# specific heat at constant volume (J/kg/K)
Cv = PropsSI('Cvmass', 'T', T, 'P', p, 'Air')
# Viscosity (Pa·s)
mu = PropsSI('V', 'T', T, 'P', p, 'Air')
# Thermal conductivity (W/m·K)
k = PropsSI('L', 'T', T, 'P', p, 'Air')

In [16]:
Cp * (J/(kg*K))

<Quantity(1029.86879, 'joule / kilogram / kelvin')>

In [17]:
gamma = Cp / Cv
gamma

In [18]:
Pr1 = Prandtl(mu, Cp, k)
Pr1

In [19]:
Pr2 = Prandtl(gamma)
Pr2

If I were to use the calorically perfect gas assumption, then $C_{p}, \, C_{v}$ are constants and would not depend on temperature. Then, the values of the Prandtl number computed with the two different methods would be essentially the same:

In [20]:
res = gas_solver("r", 287.05*J/kg/K, "gamma", 1.4)
res.show()

key     quantity         
-------------------------
gamma   gamma                 1.40000000
R       R [J / K / kg]      287.05000000
Cp      Cp [J / K / kg]    1004.67500000
Cv      Cv [J / K / kg]     717.62500000


In [21]:
k = thermal_conductivity_hansen(T*K)
k

<Quantity(0.0364091788, 'watt / meter / kelvin')>

In [22]:
mu = viscosity_air_southerland(T*K)
mu

<Quantity(2.67053335e-05, 'kilogram / meter / second')>

In [23]:
Pr1 = Prandtl(mu, res["Cp"], k)
Pr1

<Quantity(0.736907061, 'joule / second / watt')>

In [24]:
Pr2 = Prandtl(res["gamma"])
Pr2